# Marker recognition performances
Marker recognition should be done in real time during rover exploration.\
Depending on the platform used to run the code, significant different time execution has been obtained. 

In [1]:
# Import libraries once
import numpy as np
import cv2
import cv2.aruco as aruco
import matplotlib.pyplot as plt

# Constant parameters used in Aruco methods
ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_50)

### Original, colored image
First we evaluate the plain function performance, given a non-filtered colored jpg image at **full resolution**. Photos are taken from smartphone with native resolution of 1960x4032:

In [2]:
# Read the image with the markers
queryImg = cv2.imread("test-imgs/two-A4-splitted_full.jpg")
print("Resolution: {}x{}".format(queryImg.shape[0], queryImg.shape[1]))

%timeit corners, ids, _ = aruco.detectMarkers(queryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)


Resolution: 1960x4032
106 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Consideration about absolute timing cannot be provided, because it depends on the platform used.\
In Desktop PC the timeit returns an execution time of 106ms, contrariwise, on a SBC it took 227ms .

### Black/white image
We now try to do better. We consider a black/white image that is more light wrt memory occupation. We should also consider that camera image would be colored, so it is also relevant time spent for "cvtColor" function to convert the image. Image is full resolution also for this test.

In [3]:
grayQueryImg = cv2.cvtColor(queryImg, cv2.COLOR_BGR2GRAY)

%timeit corners, ids, _ = aruco.detectMarkers(grayQueryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

109 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit grayQueryImg = cv2.cvtColor(queryImg, cv2.COLOR_BGR2GRAY)

4.28 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We did not expect improvements in performance because the function perform itself a conversion to black/white internally or skips it if image is already b/w. The b/w image is useful for other processes so the conversion can be accomplished once for all.
SBC: 182ms, PC: 109ms

### Filtered image
We now try if filtering the image can speed up the search, due to the fact that less details are present in image. Also for this test, take full resolution image.

First try with a **bilateral filter**:

In [4]:
queryFiltered = cv2.bilateralFilter(queryImg, 9, 75, 75, cv2.BORDER_DEFAULT)

%timeit corners, ids, _ = aruco.detectMarkers(queryFiltered, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

156 ms ± 5.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


There is a speed up of about 31% wrt the detection without filtering, but we know that the bilateral filter is great for edge preservation but not computationally efficient. (This test was only to check the speed improvement with good filter, of course not applicable in practice)

We can try also with other filtering e.g. **Median Blur**, already implemented in cv2:

In [5]:
%timeit queryFiltered = cv2.medianBlur(queryImg, 3)

19.2 ms ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Median filter is more time friendly, we can check on field if it lead to better results with smaller tags.

## Scaled image
We now try with a scaled version of the iamge. This could be not very good for edge detection, some light edge can be masked.

In [7]:
# Make a scaled version of the original image
scale_perc = 60 # percent of original size
width = int(queryImg.shape[1] * scale_perc / 100)
height = int(queryImg.shape[0] * scale_perc / 100)
dim = (width, height)

%timeit imgResized = cv2.resize(queryImg, dim, interpolation = cv2.INTER_AREA)
imgResized = cv2.resize(queryImg, dim, interpolation = cv2.INTER_AREA)

print("Resized Dimensions :", imgResized.shape)

88.9 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Resized Dimensions : (1176, 2419, 3)


In [8]:
%timeit corners, ids, _ = aruco.detectMarkers(imgResized, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

43 ms ± 3.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


With a scaled version of the image we get a speedup of about 67%. Could be useful when operating near to the panel and marker size is sufficient for a smaller size image to be elaborated. We need to take in consideration the time that is needed to perform the scaling. If we need to perform other search we could "distribute" this time to the overall number of operations. (e.g. pattern search for detecting plugs or switches)

# Final considerations
Filtering the image not seems to be useful wrt the aruco recognition, regarding the computational time.\
Depending on the final camera specifications, the image can be upscaled or downscaled to get optimal performace.\
The scaling or filtering operations make sense if further operations need to be performed on the image, otherwise the time spent for this operations could be much longer than the plain search itself.